# DTSC670: Foundations of Machine Learning Models
## Module 2
## Assignment 4: Custom Transformer and Transformation Pipeline

#### Name: Betty Tai

Begin by writing your name above.

Your task in this assignment is to create a custom transformation pipeline that takes in raw data and returns fully prepared, clean data that is ready for model training.  However, we will not actually train any models in this assignment.  This pipeline will employ an imputer class, a user-defined transformer class, and a data-normalization class.

Please note that the order of features in the final feature matrix must be correct.  See the below figure that illustrates the input and output of the transformation pipeline.  The positions of features $x_1$ and $x_2$ do not change - they remain in the first and second columns, respectvely, both before and after the transformation pipeline.  In the transformed dataset, the $x_5$ feature is next, and is followed by the newly computed feature $x_6$.  Finally, the last two columns are the remaining one-hot vectors obtained from encoding the categorical feature $x_3$.

<img src="DataTransformation.png " width ="500" />

# Import Data

Import data from the file called `CustomTransformerData.csv`.

In [14]:
import numpy as np
import pandas as pd

df = pd.read_csv("CustomTransformerData.csv")
df_copy = df.copy()
df

,x1,x2,x3,x4,x5
0,1.5,2.354153,COLD,593,0.750000
1,2.5,3.314048,WARM,340,2.083333
2,3.5,4.021604,COLD,551,4.083333
3,4.5,NaN,COLD,2368,6.750000
4,5.5,5.847601,WARM,2636,10.083333
5,6.5,7.229910,WARM,2779,14.083333
6,7.5,7.997255,HOT,1057,18.750000
7,8.5,9.203947,COLD,819,24.083333
8,9.5,10.335348,WARM,3349,NaN
9,10.5,11.112142,HOT,3235,36.750000


# Create Custom Transformer

Create a custom transformer, just as we did in the lecture video entitled "Custom Transformers", that performs two computations: 

1. Adds an attribute to the end of the data (i.e. new last column) that is equal to $\frac{x_1^3}{x_5}$ for each observation

2. Drops the entire $x_4$ feature column.  (See further instructions below.)

You must name your custom transformer class `Assignment4Transformer`.  Your class should include a parameter with a default value of `True` that deletes the $x_4$ feature column when its value is `True`, but preserves the $x_4$ feature column when its value is `False`.

NOTE: You must handle the numeric and categorical features separately.  Accordingly, you will not pass the $x_3$ feature column through this custom transformer.  This means your calculations should reflect the absence of the $x_3$ feature column when indexing data structures.

In [15]:
dropx4 = 1
df_copy["x6"] = (df_copy["x1"]**3) / df_copy["x5"]

#X
from sklearn.base import BaseEstimator, TransformerMixin

x1_ix, x2_ix, x3_ix, x4_ix, x5_ix = 0, 1, 2, 3, 4

class Assignment4Transformer(BaseEstimator, TransformerMixin): 
    def __init__(self, dropx4 = True):
        self.dropx4 = dropx4
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        x6 = (X[:, x1_ix]**3) / X[:, x5_ix]
        if True: 
            return np.c_[X[:,0:2],X[:,4], x6]
        else: 
            return self 


# Create Transformation Pipeline for Numerical Features

Create a custom transformation pipeline for numeric data only called `num_pipeline` that:

1. Applies the `SimpleImputer` class to the data, where the strategy is set to `mean`.

2. Applies the custom `Assignment4Transformer` class to the data.

3. Applies the `StandardScaler` class to the data.

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = "mean")
x4 = Assignment4Transformer()

num_pipeline = Pipeline([
                    ('imputer', SimpleImputer(strategy = "mean")), 
                    ('transform', Assignment4Transformer()),
                    ('std_scaler', StandardScaler())])

# Create Numeric and Categorical DataFrames

Create two new data frames.  Create one DataFrame called `data_num` that holds the numeric features.  Create another DataFrame called `data_cat` that holds the categorical features.

In [19]:
data_num = df_copy.drop("x3", axis=1).copy()
data_cat = df_copy[["x3"]].copy()

data_cat

,x3
0,COLD
1,WARM
2,COLD
3,COLD
4,WARM
5,WARM
6,HOT
7,COLD
8,WARM
9,HOT


# Quick Testing

The full pipeline will be implemented with a `ColumnTransformer` class.  However, to be sure that our numeric pipeline is working properly, lets invoke the `fit_transform()` method of the `num_pipeline` object.  Then, take a look at the transformed data to be sure all is well.

### Run Pipeline and Create Transformed Numeric Data

In [21]:
# 4 columnns
data_transformed = num_pipeline.fit_transform(data_num)
data_transformed

array([[-1.63835604e+00, -1.72914963e+00, -1.64447439e+00,
        -1.16710455e+00],
       [-1.44560827e+00, -1.52555901e+00, -1.45167394e+00,
        -1.12957768e+00],
       [-1.25286050e+00, -1.37548847e+00, -1.25887349e+00,
        -1.07328737e+00],
       [-1.06011273e+00,  0.00000000e+00, -1.06607305e+00,
        -9.98233625e-01],
       [-8.67364964e-01, -9.88200404e-01, -8.73272604e-01,
        -9.04416443e-01],
       [-6.74617195e-01, -6.95017046e-01, -6.80472159e-01,
        -7.91835823e-01],
       [-4.81869425e-01, -5.32265568e-01, -4.87671714e-01,
        -6.60491767e-01],
       [-2.89121655e-01, -2.76330174e-01, -2.94871269e-01,
        -5.10384275e-01],
       [-9.63738849e-02, -3.63635906e-02,  2.28321593e-16,
        -3.86207191e-01],
       [ 9.63738849e-02,  1.28392005e-01,  9.07296210e-02,
        -1.53878980e-01],
       [ 2.89121655e-01,  2.65718106e-01,  2.83530066e-01,
         5.25188223e-02],
       [ 4.81869425e-01,  4.50133099e-01,  4.76330511e-01,
      

### One-Hot Encode Categorical Features

Similarly, you will employ a `OneHotEncoder` class in the `ColumnTransformer` below to construct the final full pipeline.  However, let's instantiate an object of the `OneHotEncoder` class called `cat_encoder` that has the `drop` parameter set to `first`.  Next, call the `fit_transform()` method and pass it your categorical data.  Take a look at the transformed one-hot vectors to be sure all is well.

In [30]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(drop='first')
data_cat_1hot = cat_encoder.fit_transform(data_cat)
data_cat_1hot

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

# Put it All Together with a Column Transformer

Now, we are finally ready to construct the full transformation pipeline called `full_pipeline` that will transform our raw data into clean, ready-to-train data.  Construct this ColumnTransformer below, then call the `fit_transform()` method to obtain the final, clean data.  Save this output data into a variable called `data_trans`.

In [38]:
from sklearn.compose import ColumnTransformer

num_attribs = list(data_num)
cat_attribs = ["x3"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs), 
    ("cat", OneHotEncoder(drop='first'), cat_attribs)])

In [39]:
data_trans = full_pipeline.fit_transform(df_copy)
data_trans

array([[-1.63835604e+00, -1.72914963e+00, -1.64447439e+00,
        -1.16710455e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.44560827e+00, -1.52555901e+00, -1.45167394e+00,
        -1.12957768e+00,  0.00000000e+00,  1.00000000e+00],
       [-1.25286050e+00, -1.37548847e+00, -1.25887349e+00,
        -1.07328737e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.06011273e+00,  0.00000000e+00, -1.06607305e+00,
        -9.98233625e-01,  0.00000000e+00,  0.00000000e+00],
       [-8.67364964e-01, -9.88200404e-01, -8.73272604e-01,
        -9.04416443e-01,  0.00000000e+00,  1.00000000e+00],
       [-6.74617195e-01, -6.95017046e-01, -6.80472159e-01,
        -7.91835823e-01,  0.00000000e+00,  1.00000000e+00],
       [-4.81869425e-01, -5.32265568e-01, -4.87671714e-01,
        -6.60491767e-01,  1.00000000e+00,  0.00000000e+00],
       [-2.89121655e-01, -2.76330174e-01, -2.94871269e-01,
        -5.10384275e-01,  0.00000000e+00,  0.00000000e+00],
       [-9.63738849e-02, -3.63635906e-02,  2.283

# Prepare for Grading

Prepare your `data_trans` NumPy array for grading by using the NumPy [around()](https://numpy.org/doc/stable/reference/generated/numpy.around.html) function to round all the values to 2 decimal places - this will return a NumPy array.

Please note the final order of the features in your final numpy array, which is given at the top of this document.

___You MUST print your final answer, which is the NumPy array discussed above, using the `print()` function!  This MUST be the only `print()` statement in the entire notebook!  Do not print anything else using the print() function in this notebook!___

In [40]:
print(np.around(data_trans,decimals=2))

[[-1.64 -1.73 -1.64 -1.17  0.    0.  ]
 [-1.45 -1.53 -1.45 -1.13  0.    1.  ]
 [-1.25 -1.38 -1.26 -1.07  0.    0.  ]
 [-1.06  0.   -1.07 -1.    0.    0.  ]
 [-0.87 -0.99 -0.87 -0.9   0.    1.  ]
 [-0.67 -0.7  -0.68 -0.79  0.    1.  ]
 [-0.48 -0.53 -0.49 -0.66  1.    0.  ]
 [-0.29 -0.28 -0.29 -0.51  0.    0.  ]
 [-0.1  -0.04  0.   -0.39  0.    1.  ]
 [ 0.1   0.13  0.09 -0.15  1.    0.  ]
 [ 0.29  0.27  0.28  0.05  0.    1.  ]
 [ 0.48  0.45  0.48  0.28  0.    1.  ]
 [ 0.67  0.76  0.67  0.52  0.    0.  ]
 [ 0.87  0.88  0.86  0.78  1.    0.  ]
 [ 1.06  0.    1.05  1.07  1.    0.  ]
 [ 1.25  1.42  1.25  1.37  0.    1.  ]
 [ 1.45  1.55  1.44  1.68  1.    0.  ]
 [ 1.64  1.71  1.63  2.02  1.    0.  ]]
